# Predicting the PageRank of the modified graph using the previously trained model

In [1]:
from mygraph import MyGraph
from helpers import *
import time
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

## Loading the dataset from numpy files

In [2]:
# loading the pagerank of modified graph as the target or label data
target_pagerank = np.load("fb_co_pages_modified_pageranks.npy", mmap_mode=None, allow_pickle=False, fix_imports=True, encoding='ASCII')
target_pagerank[0]

array([9.98297755e-05])

In [3]:
# scaling the PageRank data
target_pagerank = target_pagerank * 10000
target_pagerank[0]

array([0.99829776])

In [4]:
target_pagerank.shape

(14113, 1)

In [5]:
# the order 5 NFDC matrix as the feature set
matrix_1 = np.load("fb_co_pages_modified_NDFC_matrix_r1-30_sta1_max50_rad5.npy", mmap_mode=None, allow_pickle=False, fix_imports=True, encoding='ASCII')
matrix_1.shape

(14113, 6, 17)

In [6]:
matrix_1 = matrix_1.reshape(14113,-1)
matrix_1.shape

(14113, 102)

In [7]:
features = torch.from_numpy(matrix_1)
targets = torch.from_numpy(target_pagerank)
features.shape, targets.shape

(torch.Size([14113, 102]), torch.Size([14113, 1]))

## Defining the feedforward neural network model

In [8]:
# The Feedforward Neural Network 
class FFNN_model(nn.Module):
    def __init__(self):
        super().__init__()
        num_features = features.shape[1]
        self.fc1 = nn.Linear(num_features, 400)
        self.fc2 = nn.Linear(400, 800)
        self.fc3 = nn.Linear(800, 200)
        self.fc4 = nn.Linear(200, 64)
        self.fc5 = nn.Linear(64, 8)
        self.fc6 = nn.Linear(8, 1)
        self.dropout1 = nn.Dropout(0.4)
        self.dropout2 = nn.Dropout(0.3)
        self.dropout3 = nn.Dropout(0.5)
    
    def forward(self, X):
        X = torch.tanh(self.fc1(X))
        X = torch.relu(self.fc2(X))
        X = self.dropout1(X)
        X = torch.relu(self.fc3(X))
        X = self.dropout3(X)
        X = torch.relu(self.fc4(X))
        X = self.dropout2(X)
        X = torch.tanh(self.fc5(X))
        return self.fc6(X)

## Loading the trained model

In [9]:
# Model class must be defined somewhere
model = torch.load("trained_FFNN_model_pagerank.pt")
model.eval()
model

FFNN_model(
  (fc1): Linear(in_features=102, out_features=400, bias=True)
  (fc2): Linear(in_features=400, out_features=800, bias=True)
  (fc3): Linear(in_features=800, out_features=200, bias=True)
  (fc4): Linear(in_features=200, out_features=64, bias=True)
  (fc5): Linear(in_features=64, out_features=8, bias=True)
  (fc6): Linear(in_features=8, out_features=1, bias=True)
  (dropout1): Dropout(p=0.4, inplace=False)
  (dropout2): Dropout(p=0.3, inplace=False)
  (dropout3): Dropout(p=0.5, inplace=False)
)

In [10]:
errors = []
for i in range(targets.shape[0]):
    targ = targets[i].item()
    feat = features[i].float().view(1,1,102)
    pred = model(feat).item()
    inaccuracy = abs(1 - pred/targ) * 100
    errors.append(inaccuracy)
    if i%500 == 0:
        print(f"target {targ},    prediction: {pred}\nindex {i}:       inaccuracy: {np.round(inaccuracy, 3)}%")
        print("-----------------------------------")

target 0.9982977551369484,    prediction: 1.058740258216858
index 0:       inaccuracy: 6.055%
-----------------------------------
target 0.35261531558332776,    prediction: 0.4094126522541046
index 500:       inaccuracy: 16.107%
-----------------------------------
target 0.16349115794353675,    prediction: 0.202663853764534
index 1000:       inaccuracy: 23.96%
-----------------------------------
target 0.7372122576301666,    prediction: 0.775229811668396
index 1500:       inaccuracy: 5.157%
-----------------------------------
target 0.18379511837472276,    prediction: 0.20309288799762726
index 2000:       inaccuracy: 10.5%
-----------------------------------
target 0.4695373617655953,    prediction: 0.48312363028526306
index 2500:       inaccuracy: 2.894%
-----------------------------------
target 0.8163048969530051,    prediction: 0.8025742173194885
index 3000:       inaccuracy: 1.682%
-----------------------------------
target 0.5696588006569098,    prediction: 0.5720120668411255
ind

In [11]:
print("Average inaccuracy: ", np.round(sum(errors)/len(errors), 3))

Average inaccuracy:  9.783
